In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Iceberg Catalog Setup") \
    .getOrCreate()

print("Spark Running")
print(spark.sparkContext.getConf().getAll())
print("current catalog:", spark.catalog.currentCatalog())
print("Spark UI:", spark.sparkContext.uiWebUrl)

24/07/23 19:07:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Spark Running
[('spark.eventLog.enabled', 'true'), ('spark.driver.cores', '4'), ('spark.task.cpus', '4'), ('spark.executor.cores', '4'), ('spark.history.fs.logDirectory', '/home/iceberg/spark-events'), ('spark.app.submitTime', '1721761671044'), ('spark.sql.catalog.demo.s3.endpoint', 'http://minio:9000'), ('spark.eventLog.dir', '/home/iceberg/spark-events'), ('spark.app.id', 'local-1721761671815'), ('spark.sql.warehouse.dir', 'file:/home/iceberg/notebooks/notebooks/serve/spark-warehouse'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.executor.memory', '8g'), ('spark.submit.deployMode', 'client'), ('spark.driver.host', 'b9c7b6f17546'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/ja

In [2]:
wdi_data_df = spark.read.table("curated.world_development_indicators.data")

print(wdi_data_df.rdd.getNumPartitions())

5


In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType,StructField, StringType

schema_wdi = StructType([
               StructField('Country_Name', StringType(), True),
               StructField('Country_Code', StringType(), True),
               StructField('Indicator_Name', StringType(), True),
               StructField('Indicator_Code', StringType(), True),
               StructField('Indicator_Value', StringType(), True),
               StructField('year', StringType(), True)
             ])

emptyRDD              = spark.sparkContext.emptyRDD()
df_wdi_data_unpivoted = spark.createDataFrame(emptyRDD,schema_wdi)

# We loop through the years
# And then add the data of each year to the unpivoted dataframe
for year in range(1960, 2021):
  df_temp = (wdi_data_df
             .select(
               'Country_Name',
               'Country_Code', 
               'Indicator_Name', 
               'Indicator_Code',
               # We keep the column of the current year in the loop
               F.col(str(year)).alias('Indicator_Value')
             )
             .withColumn('year', F.lit(year)) # We add a column that contains the value of the year
            )
  # We append this year's data to the output dataframe via union()
  df_wdi_data_unpivoted = df_wdi_data_unpivoted.union(df_temp)

# Printing the number of partitions of the output dataframe
print(df_wdi_data_unpivoted.rdd.getNumPartitions())

df_wdi_data_unpivoted.printSchema()

df_wdi_data_unpivoted.show(10)

305
root
 |-- Country_Name: string (nullable = true)
 |-- Country_Code: string (nullable = true)
 |-- Indicator_Name: string (nullable = true)
 |-- Indicator_Code: string (nullable = true)
 |-- Indicator_Value: string (nullable = true)
 |-- year: string (nullable = true)



+--------------------+------------+--------------------+-----------------+----------------+----+
|        Country_Name|Country_Code|      Indicator_Name|   Indicator_Code| Indicator_Value|year|
+--------------------+------------+--------------------+-----------------+----------------+----+
|Africa Eastern an...|         AFE|Age dependency ra...|   SP.POP.DPND.OL|5.80595111963956|1960|
|Caribbean small s...|         CSS|   GDP (current US$)|   NY.GDP.MKTP.CD|1880306125.08709|1960|
|East Asia & Pacif...|         EAP|Population ages 0...|SP.POP.0014.FE.ZS|40.1022698469607|1960|
|           Euro area|         EMU|  Population, female|SP.POP.TOTL.FE.IN|       138020284|1960|
|Europe & Central ...|         ECS|Population ages 5...|SP.POP.5054.FE.5Y|6.14985530006535|1960|
|Fragile and confl...|         FCS|Population ages 8...|SP.POP.80UP.MA.5Y|0.21932284916253|1960|
|         High income|         HIC|Age dependency ra...|   SP.POP.DPND.YG| 46.445585532069|1960|
|           IDA total|        

In [4]:
# Write unpivoted dataframe to a new table partitioned by year

(df_wdi_data_unpivoted
.writeTo("wdi_serving.wdi_data_unpivoted")
.partitionedBy("year")
.options(format="iceberg", mode="overwrite")
.createOrReplace()
)


In [5]:
# We use agg() method to perform aggregations
# We use avg() from the pyspark.sql.functions module to generate the average
# We apply the avg() function on a column from the grouped dataframe
df_wdi_data_average = (df_wdi_data_unpivoted
                       .groupBy(
                         'Country_Name',
                         'Country_Code', 
                         'Indicator_Name', 
                         'Indicator_Code',
                       )
                       .agg(
                        F.avg('Indicator_Value').alias('Indicator_Average_Value')
                       )
                      )

df_wdi_data_unpivoted.unpersist()


DataFrame[Country_Name: string, Country_Code: string, Indicator_Name: string, Indicator_Code: string, Indicator_Value: string, year: string]

In [6]:
(df_wdi_data_average
.repartition("Indicator_Code")
.writeTo("wdi_serving.partitioned_average_indicators")
.partitionedBy("Indicator_Code")
.options(format="iceberg", mode="overwrite")
.createOrReplace()
)

In [7]:
df_co2_emissions= spark.read.table("curated.co2_passenger_cars_emissions")


In [8]:
# To retrieve the 100 vehicles with highest emissions per year per country
# We can use a window function
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

# We define our window
# We're partitioning by member state and by year
# We're then ordering by emissions in descending order using NEDC test
window_emissions  = Window.partitionBy("year", "MS").orderBy(desc("Enedc_g/km"))

# We add an order column based on the window we just defined
# And then we filter based on that column to retrieve the top 100 rows
# Finally, we drop the order column
df_co2_ordered = (df_co2_emissions
                  .withColumn("order", row_number().over(window_emissions))
                  .where("order <= 100")
                  .drop("order")
                 )

In [9]:
(df_co2_ordered
 .writeTo("eea_serving.highest_emissions")
 .options(mode='overwrite', format="iceberg")
 .partitionedBy("year", "MS")
)

In [10]:
spark.stop()